In [45]:
from nltk.util import ngrams
import pandas as pd
from random import shuffle, randint
import math, time

In [46]:
def hashString(text):
  return abs(hash(text)) % (10 ** 8)


**Không dùng PySpark**



> Shingles




In [47]:
class Shingles:
    def __init__(self):
      self.myShingles = []
    
    def splitToken(self, text, kGram):
      specialWords = '[\/.@]'
      text = text.lower()
      #remove special word
      for word in specialWords:
        text = text.replace(word, ' ')
      shinglesText = []
      #split to get Kgram
      nGram = self.getData(ngrams(text.split(), kGram))
      #connect all gram to 1 set
      for grams in nGram:
        if grams not in self.myShingles:
          self.myShingles.append(' '.join(grams))
        shinglesText.append(' '.join(grams))
      return shinglesText

    def getData(self, gen):
      try:
        for elem in gen:
          yield elem
      except (RuntimeError, StopIteration):
          return
    
    def hashShingles(self, arrayShingles):
      arrayHash = []
      for shingles in arrayShingles:
        arrayHash.append(hashString(shingles))
      return sorted(arrayHash)
  

In [48]:
#TEST SPLIT TOKEN
text1 = 'THIS is a foo bar sentences and. [i want to ngramize it]'
myShingles = Shingles()
shinglesSet = myShingles.splitToken(text1, 3)
hashShingles = myShingles.hashShingles(shinglesSet)
print(shinglesSet)
print(hashShingles)

['this is a', 'is a foo', 'a foo bar', 'foo bar sentences', 'bar sentences and', 'sentences and i', 'and i want', 'i want to', 'want to ngramize', 'to ngramize it']
[6284423, 22622771, 22854896, 31643269, 34007365, 64852186, 79322576, 87815386, 87855679, 97401676]


In [49]:
text2 = 'Hello my name is Tai. Nice to meet you This is a foo'
shinglesSet2 = myShingles.splitToken(text2, 3)
hashShingles2 = myShingles.hashShingles(shinglesSet)
print(shinglesSet2)
print(hashShingles2)

['hello my name', 'my name is', 'name is tai', 'is tai nice', 'tai nice to', 'nice to meet', 'to meet you', 'meet you this', 'you this is', 'this is a', 'is a foo']
[6284423, 22622771, 22854896, 31643269, 34007365, 64852186, 79322576, 87815386, 87855679, 97401676]


In [50]:
print(myShingles.myShingles)

['this is a', 'is a foo', 'a foo bar', 'foo bar sentences', 'bar sentences and', 'sentences and i', 'and i want', 'i want to', 'want to ngramize', 'to ngramize it', 'hello my name', 'my name is', 'name is tai', 'is tai nice', 'tai nice to', 'nice to meet', 'to meet you', 'meet you this', 'you this is', 'this is a', 'is a foo']


In [51]:
def createMaxtrixTrueFalse(listShingles, *args):
  data = {}
  indexDoc = 1;
  for s in args:
    columnName = 'set' + str(indexDoc)
    arrayTrueFalse = []
    for shingle in listShingles:
      if (shingle in s):
        arrayTrueFalse.append(1)
      else:
        arrayTrueFalse.append(0)
    data[columnName] = arrayTrueFalse
    indexDoc += 1
  return pd.DataFrame(data) 
  #return pd.DataFrame(data,index =listShingles)

In [52]:
#TEST CREATE MATRIX
maxtrixTrueFalse = createMaxtrixTrueFalse(myShingles.myShingles, shinglesSet, shinglesSet2)
print(maxtrixTrueFalse)

    set1  set2
0      1     1
1      1     1
2      1     0
3      1     0
4      1     0
5      1     0
6      1     0
7      1     0
8      1     0
9      1     0
10     0     1
11     0     1
12     0     1
13     0     1
14     0     1
15     0     1
16     0     1
17     0     1
18     0     1
19     1     1
20     1     1


> Min Hash


In [53]:
class MinHash:
  def __init__(self, sig_size, listShingles):
    self.sig_size = sig_size
    self.listShingles = listShingles
    self.hash_functions = []
    self.createPermuation()
    #print(self.hash_functions)
  
  #create array permutation
  def createPermuation(self):
    #create array from 1 to len(listShingles)
    for i in range(0,self.sig_size):
      arrayNumberInPermution = list(range(1,len(self.listShingles)+1))
      shuffle(arrayNumberInPermution)
      self.hash_functions.append(arrayNumberInPermution)

  def computeSetSignature(self, set_):
    set_sig = []
    maxtrixTrueFalse = createMaxtrixTrueFalse(self.listShingles, set_)
    #just get only row is 1
    maxtrixTrue = maxtrixTrueFalse[maxtrixTrueFalse['set1']==1]
    #run each column in permutation and check get value min
    for columnPermutation in self.hash_functions:
      min_hash = math.inf
      for index, row in maxtrixTrue.iterrows():
          if (columnPermutation[index] < min_hash):
            min_hash = columnPermutation[index]
      set_sig.append(min_hash)
    return set_sig

  #set Singles is array of singles each document
  def computeSignatureMatrix(self, setSingles):
    signatures = []
    for s in setSingles:
      signatures.append(self.computeSetSignature(s))
    return signatures


In [54]:
#TEST MIN HASHING
myShingles = Shingles()
shinglesSet1 = myShingles.splitToken(text1, 3)
shinglesSet2 = myShingles.splitToken(text2, 3)
minHash = MinHash(6, myShingles.myShingles)

print(minHash.computeSignatureMatrix([shinglesSet1, shinglesSet2]))

[[2, 1, 2, 2, 1, 2], [1, 1, 1, 1, 3, 1]]


In [55]:
class LSH:
  def __init__(self, threshold=0.8):
    self.threshold = threshold
  
  #create band with number of bands (bands_nr)
  #divide item in maxtrix into bands_nr of bands
  def divideMatrixToNBands(self, sigMatrix, bandsNr, signLen):
    r = int(signLen/bandsNr)#number of rows in each band
    bands = {}
    for i in range(0, bandsNr):
      bands[i] = []
    #print(sigMatrix[:10])
    for signature in sigMatrix:
      for i in range(0, bandsNr):
        idx = i*r
        for x in signature[idx:idx+r]:
          bands[i].append(str(x))
    #print(bands)
    return bands
  
  #hash item in band into buckets
  def hashItemToBucketOfBand(self, band):
    buckets = {}
    #docId is index of doc in dataset
    for docId in range(0,len(band)):
      value = hashString(band[docId])
      if value not in buckets:
        buckets[value] = [docId]
      else:
        buckets[value].append(docId)
    #print(buckets)
    return buckets

  #pair group
  def groupPairItemCandidate(self, buckets):
    candidates = set()
    #print("-----------------------------------------------")
    #print(buckets)
    for bucket, candidateList in buckets.items():
      if (len(candidateList) > 1):
        for i in range(0, len(candidateList)-1):
          for j in range(i+1, len(candidateList)):
            pair = tuple(sorted((candidateList[i], candidateList[j])))
            candidates.add(pair)
    #print("Candidates")
    #print(candidates)
    return candidates #ie a set of couples, each couple is a candidate pair
  
  #compair docs
  def checkCandidates(self, candidatesList, threshold, sigs):
    similar_docs = set()

    for similarPair in candidatesList:
      docId1 = similarPair[0]
      docId2 = similarPair[1]
      #print(doc_id_1)
      signature1 = set(sigs[docId1])
      signature2 = set(sigs[docId2])
      js = len(signature1.intersection(signature2)) / len(signature1.union(signature2))

      if js >= threshold:
        similar_docs.add(tuple(sorted((docId1, docId2))))
    return similar_docs
  
  def getSimilarItems(self, sigMatrix, bandsNr, signLen):
    similarDocs = set()
    bands = self.divideMatrixToNBands(sigMatrix, bandsNr, signLen)
    #print(bands)
    for band_id, elements in bands.items():
      buckets = self.hashItemToBucketOfBand(elements)
      candidates = self.groupPairItemCandidate(buckets)
      for sim_tuple in self.checkCandidates(candidates, self.threshold, sigMatrix):
        similarDocs.add(sim_tuple)
        
    return similarDocs

In [60]:
dataset = pd.read_csv("dataset.tsv", sep="\t")
dataset['doc_id'] = dataset.index
docNr = dataset['doc_id'].max()
print(dataset['doc_id'].max())

2626


In [56]:


print("Tranfer singles")
startTime = time.time()

shinglingList = [None] * (docNr+1)
shinglingSize = 10
signatureSize = 3
bandsNr = 10

shingles = Shingles()
for index, row in dataset.iterrows():
  doc = row['Title']+" "+row['Short Description']
  i = row['doc_id']
  shinglings = shingles.splitToken(doc, shinglingSize)
  shinglingList[i] = shinglings

#print(shingling_list[:10])
endTime = time.time()
print("Time: "+str((endTime-startTime)))

Tranfer singles
Time: 836.4759745597839


In [57]:

print("Tranfer Min Hash")
signatureSize = 10
startTime = time.time()
minHash = MinHash(signatureSize, shingles.myShingles)

signatureMatrix = minHash.computeSignatureMatrix(shinglingList) 
print(signatureMatrix[:10])
endTime = time.time()
print("Time: "+str((endTime-startTime)))


Tranfer Min Hash
[[84, 134, 93, 136, 100, 82, 57, 7, 8, 60], [7, 13, 15, 11, 25, 4, 4, 9, 16, 26], [413, 2942, 4960, 7162, 15183, 2967, 3456, 162, 6358, 3254], [10932, 2829, 4065, 34237, 6407, 8462, 937, 1966, 7072, 569], [8944, 818, 6762, 2053, 717, 104, 205, 658, 2363, 2086], [8973, 11621, 23796, 4116, 8022, 5045, 7128, 745, 30437, 1867], [12, 7248, 3856, 4168, 7769, 6012, 4517, 19202, 2641, 1689], [6670, 25653, 54002, 17724, 6837, 7391, 20634, 13995, 153, 20082], [2799, 16867, 6877, 9974, 2023, 5100, 16122, 1707, 12477, 4401], [220, 39, 1778, 559, 257, 1542, 304, 169, 340, 827]]
Time: 1966.3656435012817


In [58]:
print("Transfer LSH")
lsh = LSH(threshold =0.8)

startTime = time.time()
lshSimilarItem = lsh.getSimilarItems(signatureMatrix, bandsNr, signatureSize)
endTime = time.time()
#print(lshSimilarItem)
print("Time: "+str((endTime-startTime)))

Transfer LSH
Time: 0.8511092662811279


In [59]:
for i in range(0,3):
  docs = lshSimilarItem.pop()
  print("Doc 1: ")
  print(dataset.iloc[docs[0]])
  print("Similar Doc 2: ")
  print(dataset.iloc[docs[1]])
  print("------------------------------------------------------------------------------------")

Doc 1: 
Title                Avviato studio psicologia mette a disposizione...
Short Description    Avviato studio di psicologia mette a disposizi...
Location                                               Centro Storico 
Price (Euro)                                                      110 
Timestamp                                            9 ottobre, 18:01 
Url Adv              https://www.kijiji.it/annunci/stanze-e-posti-l...
doc_id                                                             506
Name: 506, dtype: object
Similar Doc 2: 
Title                Avviato studio psicologia mette a disposizione...
Short Description    Avviato studio di psicologia mette a disposizi...
Location                                               Centro Storico 
Price (Euro)                                                      110 
Timestamp                                            9 ottobre, 18:01 
Url Adv              https://www.kijiji.it/annunci/stanze-e-posti-l...
doc_id                      